# BioExplorer - Vasculature
![](../bioexplorer_vasculature_banner.png)

### Connect to back-end

In [13]:
from bioexplorer import BioExplorer, VasculatureDisplacementParams, Vector2, Vector3
import seaborn as sns

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

### Load vasculature

In [14]:
assembly_name = 'Vasculature'
population_name = 'vasculature'
be.remove_assembly(assembly_name)

'''Defines a clipping cylinder with an origin, direction and radius'''
o = Vector3(334.274978637695, 0, 598.822242736816)
d = Vector3(0.0, 1.0, 0.0)
r = 400.0
filter = '%s.length(%s.cross(array[%f-x,%f-y,%f-z], array[%f::double precision, %f, %f])) < %f' % (population_name, population_name, o.x, o.y, o.z, d.x, d.y, d.z, r)

vasculature_assembly = be.add_assembly(assembly_name)
vasculature_model = be.add_vasculature(
    assembly_name=assembly_name,
    population_name=population_name,
    representation=be.VASCULATURE_REPRESENTATION_OPTIMIZED_SEGMENT,
    color_scheme=be.VASCULATURE_COLOR_SCHEME_REGION,
    realism_level=be.VASCULATURE_REALISM_LEVEL_ALL,
    sql_filter=filter, displacement_params=VasculatureDisplacementParams(
        segment=Vector2(0.4, 0.5)
    )
)

### Vasculature information

In [15]:
vasculature_info = be.get_vasculature_info(assembly_name=assembly_name)
print(vasculature_info)

{'modelId': 1, 'nbNodes': 831}


### Set material

In [16]:
def set_materials(palette_name):
    model_ids = be.get_model_ids()['ids']
    model_id = model_ids[len(model_ids)-1]
    material_ids = be.get_material_ids(model_id)['ids']
    nb_material_ids = len(material_ids)
    palette = sns.color_palette(palette_name, nb_material_ids)
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    specular_exponents = list()
    reflection_indices = list()
    opacities = list()
    for _ in material_ids:
        shading_modes.append(be.SHADING_MODE_PERLIN)
        user_params.append(0.002)
        glossinesses.append(0.1)
        specular_exponents.append(5.0)
        reflection_indices.append(0.1)
        opacities.append(0.9)

    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        shading_modes=shading_modes, user_parameters=user_params,
        glossinesses=glossinesses, specular_exponents=specular_exponents,
        reflection_indices=reflection_indices, opacities=opacities,
        diffuse_colors=palette, specular_colors=palette)

set_materials('Reds_r')
be.reset_camera()
core.set_renderer()

True

### Rendering settings

In [17]:
status = core.set_renderer(
    current='bio_explorer',
    subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.gi_distance = 1000.0
params.shadows = 1.0
params.soft_shadows = 1.0
status = core.set_renderer_params(params)